In [1]:
!pip install pyltr

Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.


In [2]:
!pip install more_itertools

Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.


In [3]:
import warnings
import pyltr
warnings.filterwarnings('ignore')

//miniconda3/envs/olp-sdk-for-python-0.6.0-env/lib/python3.7/site-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)


In [4]:
from sklearn.cluster import KMeans
import numpy as np
from more_itertools import sort_together

In [5]:
## Finding the key inside a dict
def find_matching_key_index(id, data):
    for i in range(0,len(data)):
        if data[i]['id'] == id:
            return i
    return -1


## partition data cluster wise
def partition_data_cluster_wise(X_d,Y_d,T_q,labels):
    data_dict = []
    for i in range(0, len(labels)):
        idx = find_matching_key_index(labels[i],data_dict )
        if idx == -1:
            data = {}
            data['id'] = labels[i]
            data['data'] = X_d[i:i+1]
            data['label'] = Y_d[i:i+1]
            data['tqid'] = T_q[i:i+1]
            data_dict.append(data)
        else :
            data = {}
            d =  data_dict[idx]['data']
            la =  data_dict[idx]['label']
            tq =  data_dict[idx]['tqid']
            data_dict[idx]['data'] = np.concatenate((d, X_d[i:i+1]), axis=0)
            data_dict[idx]['label'] = np.concatenate((la, Y_d[i:i+1]), axis=0)
            data_dict[idx]['tqid'] = np.concatenate((tq, T_q[i:i+1]), axis=0)

    return data_dict

## concatenate all the evaluation across clusters
def concatenate_cluster_eval(p_data):
    m_data = {}
    m_data['data'] = np.array([[np.array(xx)] for xx in  p_data[0]['eval']])
    m_data['label'] = p_data[0]['label']
    m_data['tqid'] = p_data[0]['tqid']
    for j in range(1, len(p_data)):
        dd = np.array([[np.array(xx)] for xx in  p_data[j]['eval']])
        m_data['data'] = np.concatenate((m_data['data'] ,dd), axis=0)
        m_data['label'] = np.concatenate((m_data['label'] , p_data[j]['label']), axis=0)
        m_data['tqid'] = np.concatenate((m_data['tqid'] , p_data[j]['tqid']), axis=0)
    return m_data

## train model using LamdaMart 
def train_model_using_LambdaMART(p_data):
    for j in range(0,len(p_data)):
        metric = pyltr.metrics.NDCG(k=5)
    
        model = pyltr.models.LambdaMART(
            metric=metric,
            n_estimators=500,
            verbose=1,
        )

        model.fit(p_data[j]['data'], p_data[j]['label'], p_data[j]['tqid'])
        p_data[j]['model'] = model
        p_data[j]['metric'] = metric
      
    return p_data

##evaluate partiotioned data
def evaluate_partitioned_data(p_data) :
    for j in range(0,len(p_data)):
        Epred = p_data[j]['model'].predict(p_data[j]['data'])
        evl_arr = evalute_preds(p_data[j]['label'],Epred,p_data[j]['tqid'], p_data[j]['metric'])
        p_data[j]['eval'] = evl_arr
    return p_data

##evaluate on all features
def evaluate_features(p_data, X, Y, tqids):
    eval_dict_arr = []
    eval_dict = {}
    for j in range(0,len(p_data)):
        Epred = p_data[j]['model'].predict(X)
        evl_arr = evalute_preds(Y, Epred, tqids, p_data[j]['metric'])
        eval_dict['id'] = j
        eval_dict['features'] = X
        eval_dict['eval'] = evl_arr
        eval_dict_arr.append(eval_dict)
    return eval_dict_arr
    
"""
Evaluate prediction
"""
def evalute_preds(Y,Y_pred,Qids, met):
    ev_arr = []
    query_groups = pyltr.util.group.get_groups(Qids)
    for qd, a,b in query_groups:
        m = met.evaluate_preds(qd,Y[a:b], Y_pred[a:b] )
        #copy the same value b-a times
        ev_arr.extend([m]*(b-a))
    return  ev_arr

"""
Group data inside a cluster based on ids as ids needs to be  contiguous.
It may happen that after clustering the tids are not contigous then LambdaMart will throw error
"""
def group_tqids(p_data):
    for j in range(0, len(p_data)):
        aa = p_data[j]['data']
        bb = p_data[j]['label']
        cc = p_data[j]['tqid']
        ord_data = sort_together([cc,aa.flatten(),bb])
        p_data[j]['data'] = np.array([[np.array(d)] for d in ord_data[1]])
        p_data[j]['label'] = np.array(ord_data[2])
        p_data[j]['tqid'] = np.array(ord_data[0])
    return p_data

In [6]:
with open('./MQ2007/Fold1/train.txt') as trainfile, \
         open('./MQ2007/Fold1/vali.txt') as valifile, \
         open('./MQ2007/Fold1/test.txt') as evalfile:
    TX, Ty, Tqids, _ = pyltr.data.letor.read_dataset(trainfile)
    VX, Vy, Vqids, _ = pyltr.data.letor.read_dataset(valifile)
    EX, Ey, Eqids, _ = pyltr.data.letor.read_dataset(evalfile)

In [27]:
Tqids

array(['10', '10', '10', ..., '6000', '6000', '6000'], dtype='<U4')

### Train Base Line model on all features

In [7]:
base_metric = pyltr.metrics.NDCG(k=5)
    
base_model = pyltr.models.LambdaMART(
            metric=base_metric,
            n_estimators=500,
            verbose=1,
)

base_model.fit(TX,Ty, Tqids)

 Iter  Train score    Remaining                           Monitor Output 
    1       0.1600        6.02m                                         
    2       0.1986        6.29m                                         
    3       0.2943        6.45m                                         
    4       0.3486        6.54m                                         
    5       0.3530        6.64m                                         
    6       0.3555        6.69m                                         
    7       0.3613        6.74m                                         
    8       0.3678        6.75m                                         
    9       0.3696        6.79m                                         
   10       0.3730        6.81m                                         
   15       0.3894        6.93m                                         
   20       0.4031        7.01m                                         
   25       0.4104        7.14m                   

#### Step 1, take the average of the features

In [9]:
##Step 1
## compute the average of the feature
NO_OF_CLUSTER=10
f_avg = np.array([[np.mean(subarray)] for subarray in TX])
##cluster the feature into 5 clusters as stated in the research paper
km_f_avg = KMeans(n_clusters= NO_OF_CLUSTER, random_state=0).fit(f_avg)
km_f_avg.labels_

array([7, 8, 4, ..., 5, 7, 3], dtype=int32)

#### Train the model on each cluster

In [10]:
## Arrange the data cluster wise
partitioned_data = partition_data_cluster_wise(f_avg,Ty,Tqids,km_f_avg.labels_)
##train model for each cluster
partitioned_data = train_model_using_LambdaMART(partitioned_data)

 Iter  Train score    Remaining                           Monitor Output 
    1       0.0744       20.63s                                         
    2       0.0838       18.68s                                         
    3       0.0848       18.15s                                         
    4       0.0847       17.69s                                         
    5       0.0840       17.58s                                         
    6       0.0859       17.55s                                         
    7       0.0864       17.74s                                         
    8       0.0849       17.86s                                         
    9       0.0864       17.85s                                         
   10       0.0864       17.78s                                         
   15       0.0881       17.31s                                         
   20       0.0889       17.16s                                         
   25       0.0889       17.32s                   

  140       0.4312       25.81s                                         
  160       0.4347       24.36s                                         
  180       0.4383       22.90s                                         
  200       0.4421       21.44s                                         
  220       0.4447       20.02s                                         
  240       0.4490       18.58s                                         
  260       0.4521       17.16s                                         
  280       0.4544       15.71s                                         
  300       0.4585       14.28s                                         
  320       0.4600       12.84s                                         
  340       0.4605       11.40s                                         
  360       0.4627        9.98s                                         
  380       0.4640        8.55s                                         
  400       0.4668        7.13s                    

    8       0.4027       31.05s                                         
    9       0.4096       31.42s                                         
   10       0.4133       31.60s                                         
   15       0.4162       31.70s                                         
   20       0.4206       31.69s                                         
   25       0.4252       31.23s                                         
   30       0.4274       30.76s                                         
   35       0.4306       30.44s                                         
   40       0.4332       29.84s                                         
   45       0.4331       29.73s                                         
   50       0.4324       29.39s                                         
   60       0.4345       28.24s                                         
   70       0.4380       27.38s                                         
   80       0.4414       26.51s                    

  300       0.4653       16.78s                                         
  320       0.4648       15.08s                                         
  340       0.4663       13.40s                                         
  360       0.4696       11.74s                                         
  380       0.4731       10.07s                                         
  400       0.4754        8.39s                                         
  420       0.4761        6.73s                                         
  440       0.4789        5.07s                                         
  460       0.4798        3.38s                                         
  480       0.4815        1.70s                                         
  500       0.4834        0.00s                                         
 Iter  Train score    Remaining                           Monitor Output 
    1       0.2042       56.76s                                         
    2       0.2010       56.89s                   

#### Evaluate partitioned data and create new features set based in the evaluation

In [11]:
##evaluate partiotioned data
partitioned_data = evaluate_partitioned_data(partitioned_data)
##merge partitioned data as mentioned in the paper [q11,q12, ..q1n, q21,q22, ...., q41,...]
merg_data = concatenate_cluster_eval(partitioned_data)

### Step 2 cluster data on new feature set created from evaluation and train it again

In [12]:
km_f_avg = KMeans(n_clusters=NO_OF_CLUSTER, random_state=0).fit(merg_data['data'])
km_f_avg.labels_

array([0, 0, 0, ..., 0, 0, 0], dtype=int32)

In [19]:
partitioned_data

[{'id': 1,
  'data': array([[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]]),
  'label': array([0., 0., 0., ..., 0., 1., 0.]),
  'tqid': array(['10', '10', '10', ..., '992', '992', '992'], dtype='<U4'),
  'model': <pyltr.models.lambdamart.LambdaMART at 0x1a14a4ea50>,
  'metric': <pyltr.metrics.dcg.NDCG at 0x1034a0bd0>,
  'eval': [0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,


In [13]:
partitioned_data = partition_data_cluster_wise(merg_data['data'],merg_data['label'],merg_data['tqid'],km_f_avg.labels_)
##group the ids as it needs to be contogous
partitioned_data = group_tqids(partitioned_data)
##train model again on this dataset
partitioned_data = train_model_using_LambdaMART(partitioned_data)

 Iter  Train score    Remaining                           Monitor Output 
    1       0.0000       37.88s                                         
    2       0.0000       34.74s                                         
    3       0.0000       33.63s                                         
    4       0.0000       33.30s                                         
    5       0.0000       33.25s                                         
    6       0.0000       32.99s                                         
    7       0.0000       32.52s                                         
    8       0.0000       32.46s                                         
    9       0.0000       32.25s                                         
   10       0.0000       32.18s                                         
   15       0.0000       31.68s                                         
   20       0.0000       31.22s                                         
   25       0.0000       30.85s                   

  140       0.2697       16.06s                                         
  160       0.2695       15.08s                                         
  180       0.2683       14.24s                                         
  200       0.2684       13.30s                                         
  220       0.2664       12.39s                                         
  240       0.2658       11.47s                                         
  260       0.2662       10.54s                                         
  280       0.2680        9.73s                                         
  300       0.2677        8.81s                                         
  320       0.2688        7.92s                                         
  340       0.2680        7.03s                                         
  360       0.2694        6.13s                                         
  380       0.2684        5.25s                                         
  400       0.2684        4.36s                    

    8       0.2798       13.10s                                         
    9       0.2750       13.17s                                         
   10       0.2770       13.05s                                         
   15       0.2798       12.61s                                         
   20       0.2767       12.65s                                         
   25       0.2724       12.48s                                         
   30       0.2814       12.26s                                         
   35       0.2764       12.02s                                         
   40       0.2800       11.85s                                         
   45       0.2781       11.69s                                         
   50       0.2794       11.59s                                         
   60       0.2744       11.28s                                         
   70       0.2774       11.04s                                         
   80       0.2756       10.73s                    

  300       0.1789        4.34s                                         
  320       0.1773        3.89s                                         
  340       0.1791        3.45s                                         
  360       0.1777        3.00s                                         
  380       0.1773        2.57s                                         
  400       0.1785        2.14s                                         
  420       0.1777        1.70s                                         
  440       0.1782        1.27s                                         
  460       0.1777        0.85s                                         
  480       0.1779        0.42s                                         
  500       0.1773        0.00s                                         
 Iter  Train score    Remaining                           Monitor Output 
    1       0.1112        5.76s                                         
    2       0.1131        5.45s                   

#### Evaluate partitioned data and create new feature set based in second clusters

In [14]:
##evaluate partiotioned data
partitioned_data = evaluate_partitioned_data(partitioned_data)
##merge partitioned data as mentioned in the paper [q11,q12, ..q1n, q21,q22, ...., q41,...]
merg_data = concatenate_cluster_eval(partitioned_data)

#### Step 3 cluster data for the last time  on new feature set created from evaluation and train it again

In [15]:
km_f_avg = KMeans(n_clusters=NO_OF_CLUSTER, random_state=0).fit(merg_data['data'])
partitioned_data = partition_data_cluster_wise(merg_data['data'],merg_data['label'],merg_data['tqid'],km_f_avg.labels_)
##group the ids as it needs to be contogous
partitioned_data= group_tqids(partitioned_data)
##train model again on this dataset
partitioned_data = train_model_using_LambdaMART(partitioned_data)

 Iter  Train score    Remaining                           Monitor Output 
    1       0.0000        3.03m                                         
    2       0.0000        2.95m                                         
    3       0.0000        2.94m                                         
    4       0.0000        2.92m                                         
    5       0.0000        2.90m                                         
    6       0.0000        2.88m                                         
    7       0.0000        2.88m                                         
    8       0.0000        2.86m                                         
    9       0.0000        2.86m                                         
   10       0.0000        2.84m                                         
   15       0.0000        2.80m                                         
   20       0.0000        2.76m                                         
   25       0.0000        2.74m                   

  140       0.0560        8.10s                                         
  160       0.0550        7.62s                                         
  180       0.0550        7.14s                                         
  200       0.0550        6.69s                                         
  220       0.0550        6.25s                                         
  240       0.0550        5.78s                                         
  260       0.0550        5.31s                                         
  280       0.0560        4.85s                                         
  300       0.0550        4.39s                                         
  320       0.0550        3.97s                                         
  340       0.0550        3.52s                                         
  360       0.0550        3.08s                                         
  380       0.0550        2.65s                                         
  400       0.0550        2.21s                    

    7       0.3283       15.11s                                         
    8       0.3283       15.12s                                         
    9       0.3283       15.17s                                         
   10       0.3283       15.14s                                         
   15       0.3283       14.96s                                         
   20       0.3287       14.61s                                         
   25       0.3283       14.18s                                         
   30       0.3272       13.99s                                         
   35       0.3283       13.85s                                         
   40       0.3287       13.70s                                         
   45       0.3283       13.58s                                         
   50       0.3283       13.34s                                         
   60       0.3276       12.78s                                         
   70       0.3287       12.39s                    

  280       0.9759        0.92s                                         
  300       0.9759        0.84s                                         
  320       0.9759        0.75s                                         
  340       0.9759        0.67s                                         
  360       0.9759        0.58s                                         
  380       0.9759        0.50s                                         
  400       0.9759        0.42s                                         
  420       0.9759        0.33s                                         
  440       0.9759        0.25s                                         
  460       0.9759        0.17s                                         
  480       0.9759        0.08s                                         
  500       0.9759        0.00s                                         
 Iter  Train score    Remaining                           Monitor Output 
    1       0.1874       22.06s                   

In [16]:
##evaluate partiotioned data
partitioned_data = evaluate_partitioned_data(partitioned_data)

### Prepare training data for cluster classification

In [17]:
for k in range(0, NO_OF_CLUSTER):
    Epred = partitioned_data[k]['model'].predict(TX)
    print ('Random ranking:',  partitioned_data[k]['metric'].calc_mean_random(Tqids, Ty))
    print ('Our model:', partitioned_data[k]['metric'].calc_mean(Tqids, Ty, Epred))

Random ranking: 1244872974576130.8
Our model: 14287055213287.303
Random ranking: 0.23860252156741646
Our model: 0.14774936710209613
Random ranking: 0.22864395387641429
Our model: 0.18844250435920487
Random ranking: 0.2701862200084966
Our model: 0.006123011371784732
Random ranking: 0.2907123444763878
Our model: 0.17734531648158927
Random ranking: 0.2535434737307281
Our model: 0.006304136195454159
Random ranking: 0.2510865298219504
Our model: 0.18367960165293487
Random ranking: 0.2395402304446158
Our model: 0.00523807761932423
Random ranking: 0.24141174896379466
Our model: 0.04779460042867932
Random ranking: 0.2345688661111453
Our model: 0.13058019560532164


In [18]:
partitioned_data

[{'id': 1,
  'data': array([[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]]),
  'label': array([0., 0., 0., ..., 0., 1., 0.]),
  'tqid': array(['10', '10', '10', ..., '992', '992', '992'], dtype='<U4'),
  'model': <pyltr.models.lambdamart.LambdaMART at 0x1a14a4ea50>,
  'metric': <pyltr.metrics.dcg.NDCG at 0x1034a0bd0>,
  'eval': [0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
